In [ ]:
from pathlib import Path
import pandas as pd
import os

DATASET_FILE = Path("data", "pre_generated_data.parquet")
if not os.path.exists(DATASET_FILE):
    raise FileNotFoundError(f"File {DATASET_FILE} does not exist.")
df = pd.read_parquet(DATASET_FILE)
print(f"Loaded {len(df)} samples from {DATASET_FILE}.")
print("Number of biased features:", df.shape[1] - 3)  # Exclude label column
print("Unique biased labels: ", df.iloc[:, -2].value_counts().to_dict())
print("Unique fair labels: ", df.iloc[:, -1].value_counts().to_dict())
print(f"Number of unique biased features: {len(set(df.iloc[:, 1:-2].values.flatten()))}, number of total biased features: {len(df.iloc[:, 1:-2].values.flatten())}")
print("Biggest biased feature value:", df.iloc[:, 1:-2].values.flatten().max())
print("Smallest biased feature value:", df.iloc[:, 1:-2].values.flatten().min())
print(df.head())